In [1]:
%matplotlib inline
import random

import pandas as pd
from pandas import Series,DataFrame
import numpy as np

from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pylab as plot
params = { 
    "axes.labelsize": "large",
    "xtick.labelsize" : "x-large",
    "legend.fontsize": 20,
    "figure.dpi": 150,
    "figure.figsize" : [25, 7]
}
plot.rcParams.update(params)

In [3]:
data = pd.read_csv('News_Final.csv')
test_data = pd.read_csv('News_Test.csv') 

data.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,LinkedIn
0,99248.0,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,4/2/02 0:00,0.000000,-0.053300,-1,-1,-1
1,10423.0,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,9/20/08 0:00,0.208333,-0.156386,-1,-1,-1
2,18828.0,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,1/28/12 0:00,-0.425210,0.139754,-1,-1,-1
3,27788.0,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,3/1/15 0:06,0.000000,0.026064,-1,-1,-1
4,27789.0,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,3/1/15 0:11,0.000000,0.141084,-1,-1,-1


In [4]:
data.tail()

,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,LinkedIn
91225,60337.0,San Diego economy slips in May,San Diego County's economy slowed slightly in ...,The San Diego Union-Tribune,economy,6/30/16 23:47,0.000000,0.177667,-1,3,1
91226,103879.0,Durbin: Obama pal's ties to for-profit univers...,But now that one of President Barack Obama's c...,Chicago Tribune,obama,6/30/16 23:49,0.000000,0.110773,15,1,4
91227,60407.0,German economy minister: EU stability pact nee...,German Economy Minister Sigmar Gabriel has cal...,Reuters via Yahoo! Finance,economy,6/30/16 23:59,0.072169,-0.145833,-1,0,0
91228,60593.0,The Gig Economy's Growing Influence on the Ame...,There's been much debate about the rise of the...,Business News Daily,economy,7/1/16 0:00,-0.125000,0.027518,-1,0,29
91229,60597.0,This Is How Hillary Clinton Should Respond to ...,There's only one right move for the U.S. econo...,Fortune,economy,7/1/16 0:00,0.104167,-0.125574,-1,3,3


In [5]:
type(data.index)

pandas.core.indexes.range.RangeIndex

In [6]:
data.isnull().sum()

IDLink                 0
Title                  0
Headline              14
Source               279
Topic                  0
PublishDate            0
SentimentTitle         0
SentimentHeadline      0
Facebook               0
GooglePlus             0
LinkedIn               0
dtype: int64

In [7]:
data.shape

(91230, 11)